In [2]:
import os
import pandas as pd
import chardet  
import re  

In [16]:
# Twitter Sentiment Dataset
""" https://www.kaggle.com/datasets/kazanova/sentiment140/ """

' https://www.kaggle.com/datasets/kazanova/sentiment140/ '

In [7]:
# Get the encoding for the twitter file
%%script false

# Detect the encoding
with open('./Twitter Sentiment Data.csv', 'rb') as file:
    result = chardet.detect(file.read())
    encoding = result['encoding']

# Write encoding to a file
with open('encoding.txt', 'w') as f:
    f.write(encoding)

UsageError: Line magic function `%%script` not found.


In [18]:
# Twitter Dataset 1.6 Million Tweets
# Put the tweet and target columns into a new file

#read the encoding from the file
with open('encoding.txt', 'r') as f:
    encoding = f.read().strip()

# Read the file with the detected encoding for the two important columns
twitter_data = pd.read_csv('./Twitter Sentiment Data.csv', encoding=encoding, header=None)
twitter_data.info() # Get Twitter Data Info

# Get series of the tweets and targets
x_data = twitter_data.iloc[:, -1] # tweets
x_data.name = 'tweet'

y_targets = twitter_data.iloc[:, 0] # target value (0, 4)
y_targets.name = 'target'

x_data.head()
y_targets.head()

new_twitter_data = pd.concat([x_data, y_targets], axis=1)
new_twitter_data.to_csv('twitter_data_1600000.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   0       1600000 non-null  int64 
 1   1       1600000 non-null  int64 
 2   2       1600000 non-null  object
 3   3       1600000 non-null  object
 4   4       1600000 non-null  object
 5   5       1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [19]:
# Get the range of dates from the tweets
# earliest_date = twitter_data['date'].min()
# latest_date = twitter_data['date'].max()
# print(f"Range: {earliest_date} - {latest_date}")

In [20]:
# Get the unique hashtags from each tweet and put it in the dataframe

# Function to extract hashtags from a single tweet
def extract_hashtags(tweet):
    
    # Find all the hashtags in the tweet (words that begin with '#')
    return re.findall(r"#(\w+)", tweet)

# Create a column which the unique hastags in each tweet
twitter_data['hashtags'] = twitter_data['text'].apply(extract_hashtags)

# Create a list of all the hashtags
all_hashtags = sum(twitter_data['hashtags'].tolist(), [])

# Create a series with the unquie hashtags and their counts
hashtag_series = pd.Series(all_hashtags)
hashtag_counts = hashtag_series.value_counts()

print(hashtag_counts)

KeyError: 'text'

In [ ]:
mask = twitter_data['hashtags'].apply(lambda x: len(x) > 0)
tweets_with_hashtags = twitter_data[mask]

tweets_with_hashtags.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35847 entries, 83 to 1599999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   target    35847 non-null  int64 
 1   ids       35847 non-null  int64 
 2   date      35847 non-null  object
 3   flag      35847 non-null  object
 4   user      35847 non-null  object
 5   text      35847 non-null  object
 6   hashtags  35847 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.2+ MB


In [3]:
# Preprocess Elon Musk's twitter data
# Combine the date and tweets columns from each file into a new file

# Base directory 
base_directory = './Elon Musk Tweet Data/'

# Initialize a list to hold all the file paths
file_paths = []

# Loop over the range of years to construct file names
for year in range(2010, 2023):
    file_path = os.path.join(base_directory, f"{year}.csv") # Construct the file path
    file_paths.append(file_path) # Append the file path to the list

# print(file_paths, len(file_paths))

# Create a dataframe for all values
elon_tweets = pd.DataFrame(columns=['date', 'tweet'])

# Loop list of files
for file in file_paths:
    df = pd.read_csv(file) # Read file
    data = df[['date', 'tweet']] # Get the data and tweet columns
    elon_tweets = pd.concat([elon_tweets, data], ignore_index=True) # append to new dataframe

# Drop the repeating tweets, certain files have them
elon_tweets.drop_duplicates(inplace=True)

# Create a new file with all unique tweets from 2010-2022
elon_tweets.to_csv('elon_musk_tweets_2010-2022.csv', index=False)